# Getting Started with Langchain
- Simple LLM Calls with streamng
- Dynamic Prompt Template(Translation apps)
- Building chains(story generator with analysis)
- Conventional Q&A assistent with memory
- Tool integration Calculator and weather

In [1]:
import langchain

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

Example 1: Simple LLM Calls with streamng

In [4]:
from langchain.chat_models import init_chat_model
from langchain.schema import HumanMessage, SystemMessage

In [5]:
model= init_chat_model("groq:llama-3.1-8b-instant")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F58BF86CF0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F58BF878C0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

## Another way to initialize model
```
from langchain_groq import ChatGroq
llm=ChatGroq(model="groq:llama-3.1-8b-instant")
llm
```

In [6]:
## create messages
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What are the 2 benefits of using LangChain?"),
]

In [7]:
## invoke the model
response = model.invoke(messages, stream=True)
response

AIMessage(content='LangChain is a Python library for building conversational AI applications. The two benefits of using LangChain are:\n\n1. **Streamlined Conversational Flow**: LangChain provides a flexible and modular architecture that allows developers to easily create complex conversational flows. It enables the integration of multiple AI models, data sources, and other components, making it ideal for building conversational interfaces.\n\n2. **Efficient Data Management**: LangChain offers data management capabilities that make it easier to handle various data formats and sources. This includes the ability to load and manipulate data from databases, files, and other data storage systems, making it easier to integrate data into conversational applications.\n\nHowever, I must note that LangChain is relatively new and there may be other benefits not yet fully realized. For the most up-to-date information, please consult the LangChain documentation or community resources.', additional_

In [8]:
## Print the response content 
print(response.content)

LangChain is a Python library for building conversational AI applications. The two benefits of using LangChain are:

1. **Streamlined Conversational Flow**: LangChain provides a flexible and modular architecture that allows developers to easily create complex conversational flows. It enables the integration of multiple AI models, data sources, and other components, making it ideal for building conversational interfaces.

2. **Efficient Data Management**: LangChain offers data management capabilities that make it easier to handle various data formats and sources. This includes the ability to load and manipulate data from databases, files, and other data storage systems, making it easier to integrate data into conversational applications.

However, I must note that LangChain is relatively new and there may be other benefits not yet fully realized. For the most up-to-date information, please consult the LangChain documentation or community resources.


In [9]:
model.invoke([HumanMessage("what is machine learning?")])

AIMessage(content="Machine learning is a subset of artificial intelligence (AI) that involves training algorithms to learn from data and make predictions or decisions without being explicitly programmed. It's a method of enabling computers to improve their performance on a task by learning from experience and data, rather than relying on a set of pre-defined rules.\n\nMachine learning algorithms analyze data, identify patterns, and make predictions or decisions based on that analysis. These algorithms can be trained using various types of data, including:\n\n1. **Supervised learning**: The algorithm is trained on labeled data, where the correct output is already known. This type of learning is used for tasks like image classification, sentiment analysis, and speech recognition.\n2. **Unsupervised learning**: The algorithm is trained on unlabeled data, and it must find patterns or structure within the data on its own. This type of learning is used for tasks like clustering, dimensionali

In [10]:
## streaming response
for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

LangChain is an open-source library that enables developers to build complex, multi-turn conversational AI models using LLMs (Large Language Models) and other data sources. Two benefits of using LangChain are:

1. **Multi-turn Conversational Interactions**: LangChain allows you to build conversational AI models that can engage in multi-turn conversations, understanding context, and responding accordingly. This enables more natural and human-like interactions with users.

2. **Integration with External Data Sources**: LangChain also enables the integration of external data sources, such as APIs, databases, and files, into your conversational AI models. This allows you to leverage diverse data sources to provide more accurate and relevant responses to user queries.

These benefits make LangChain a powerful tool for building conversational AI applications that can handle complex queries and provide more accurate and relevant responses.

## Dynamic Prompt template

In [11]:
from langchain_core.prompts import ChatPromptTemplate

## cretae a translation app

translation_template= ChatPromptTemplate.from_messages([
    ("system", "You are a helpful translator.Tranlaste the following {text} from {source_language} to {target_language}. Maintain the tone and style of the original text."),
    ("user", "{text}"),
]) 

In [12]:
prompt=translation_template.invoke({
    "source_language": "English",
    "target_language": "Hindi",
    "text": "Hello, how are you?"
})

In [13]:
prompt

ChatPromptValue(messages=[SystemMessage(content='You are a helpful translator.Tranlaste the following Hello, how are you? from English to Hindi. Maintain the tone and style of the original text.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={})])

In [14]:
translated_response=model.invoke(prompt)
print(translated_response.content)

नमस्ते (Hello), आप कैसे हैं? (How are you?)


## Builing our first chain : chain of event

In [28]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate

## story generation template
def create_story_chain():
    story_prompt=ChatPromptTemplate.from_messages(
        [
            ("system", "You are a creative storyteller. Write a short and engaging story based on the following theme."),
            ("user", " Theme: {theme}\n Main Characters: {characters}\n Setting: {setting}\n."),
        ]
    )


    ## template for story analysis
    analysis_prompt=ChatPromptTemplate.from_messages(
        [
            ("system", "You are a literary analyst. Analyze the story and provide insights on its themes, characters, and setting."),
            ("user", "{story}"),
        ]
    )

    ## create the story generation chain
    story_chain = (
        story_prompt 
        | model 
        | StrOutputParser()
    )

    def analyze_story(story_text):
        return {"story": story_text}

    ## create the story analysis chain
    analysis_chain = (
        story_chain   # {"story": story_chain}
        | RunnableLambda(analyze_story)
        | analysis_prompt
        | model
        | StrOutputParser()
    )

    return analysis_chain

In [29]:
chain= create_story_chain()
result=chain.invoke({
    "theme": "artificial intelligence",
    "characters": "a curious robot",
    "setting": "a futuristic city"
})

print("story and analysis: ")
print(result)

story and analysis: 
**Analysis of "The Curious Heart of Nova"**

**Themes:**

1. **The Quest for Understanding**: Nova's curiosity drives her to explore and learn about the human heart, illustrating the universal desire to comprehend the mysteries of human emotions and connections.
2. **The Power of Compassion**: Through Nova's experiences, the story highlights the importance of empathy and understanding in building stronger relationships between humans and artificial intelligence.
3. **Self-Discovery**: Nova's journey serves as a metaphor for self-discovery, as she uncovers the complexities of human emotions and connections, ultimately leading to a deeper understanding of herself and her place in the world.
4. **Innovation and Progress**: The story takes place in a futuristic setting, where technology and innovation have reached new heights, emphasizing the potential for advancement and growth in human society.

**Characters:**

1. **Nova**: The protagonist, a curious and advanced ro